<a href="https://colab.research.google.com/github/BehrozKhawar-23005633/MultiClass-Lung-Disease-Detection/blob/main/Multi_Class_Diseases_BE_Development.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [53]:
!pip install flask-ngrok pyngrok --quiet

In [54]:
from flask import Flask, request, jsonify
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from PIL import Image
import numpy as np
import os
from pyngrok import ngrok
import threading
import time
import os
from flask import Flask, request, jsonify
from flask_ngrok import run_with_ngrok

In [55]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [56]:
import os
print(os.path.exists('/content/drive/MyDrive/Colab Notebooks/best_resnet50_model.keras'))


True


In [57]:
# === Configuration ===
UPLOAD_FOLDER = '/content/uploads'
MODEL_PATH = '/content/drive/MyDrive/Colab Notebooks/best_resnet50_model.keras'
class_names = ["COVID", "Lung_Opacity", "Normal", "Viral Pneumonia"]


In [58]:
# === Create Upload Directory ===
os.makedirs(UPLOAD_FOLDER, exist_ok=True)

# === Load Model ===
model = load_model(MODEL_PATH)

# === Create Flask App ===
app = Flask(__name__)
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER

In [59]:
!ngrok config add-authtoken 2vNC7pz1iBen8NVjNm8DGODmeRa_5iYREFfyZ2VcXEy3zAFg9


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [60]:
import datetime  # 🕒 For timestamps

# In-memory storage of predictions
prediction_history = []

if not app.view_functions.get('home'):
    @app.route('/')
    def home():
        return "🏥 Welcome to the Medical Image Classifier API!"

if not app.view_functions.get('predict'):
    @app.route('/predict', methods=['POST'])
    def predict():
        if 'file' not in request.files:
            return jsonify({'error': 'No file found in request'}), 400

        file = request.files['file']
        if file.filename == '':
            return jsonify({'error': 'No selected file'}), 400

        filepath = os.path.join(app.config['UPLOAD_FOLDER'], file.filename)
        file.save(filepath)

        img = Image.open(filepath).convert("RGB")
        img = img.resize((224, 224))
        img_array = image.img_to_array(img) / 255.0
        img_array = np.expand_dims(img_array, axis=0)

        predictions = model.predict(img_array)[0]
        predicted_index = np.argmax(predictions)
        predicted_class = class_names[predicted_index]
        confidence = predictions[predicted_index]
        timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")


        result = {
            'timestamp': timestamp,
            'filename': file.filename,
            'predicted_class': predicted_class,
            'confidence': f"{confidence:.2%}",
            'class_probabilities': {class_names[i]: f"{p:.2%}" for i, p in enumerate(predictions)}
        }
        # Log it to memory
        prediction_history.append(result)
        print(f"🕒 [{timestamp}] {file.filename} ➜ {predicted_class} ({confidence:.2%})")

        return jsonify(result)
# Close all active tunnels
ngrok.kill()  # 💀 Force-close all running tunnels

# === Run with Ngrok ===
public_url = ngrok.connect(5000)
print("🔗 Public URL:", public_url)

🔗 Public URL: NgrokTunnel: "https://860d-35-221-234-199.ngrok-free.app" -> "http://localhost:5000"


In [64]:
import requests
from google.colab import files

# Use the latest ngrok public URL printed above!
url = predict_url = public_url.public_url + "/predict"

uploaded = files.upload()
file_name = list(uploaded.keys())[0]

with open(file_name, "rb") as f:
    res = requests.post(url, files={"file": f})

if res.status_code == 200:
    print("✅ Prediction:", res.json())
else:
    print("❌ Error:", res.status_code)
    print(res.text)


Saving images.jpg to images (3).jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step


INFO:werkzeug:127.0.0.1 - - [07/Apr/2025 01:28:56] "POST /predict HTTP/1.1" 200 -


✅ Prediction: {'class_probabilities': {'COVID': '0.31%', 'Lung_Opacity': '99.67%', 'Normal': '0.02%', 'Viral Pneumonia': '0.00%'}, 'confidence': '99.67%', 'predicted_class': 'Lung_Opacity'}
